# Metadata

**L1 Taxonomy** - Software Architecture & Design

**L2 Taxonomy** - CQRS (Command Query Responsibility Segregation)

**Subtopic** - Logging all commands that happen and showing how the system state can be rebuilt by processing the log proving the concept of event sourcing typically paired with CQRS

**Use Case** - Implement a Python module that simulates a system using the CQRS pattern. This module should include two distinct parts: the Command model and the Query model. The Command model will be responsible for handling all system state changes, logging each command it processes. The Query model will be responsible for reading the state of the system. The state of the system can be rebuilt by processing the log created by the Command model. Use Python's built-in logging module to log all commands and use the JSON module to store the system state in a local JSON file.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

# Prompt

You're incharge of implementing an order management system for a pizza restaurant that separates order processing from analytics dashboard updates. The system processes pizza orders and maintains real-time kitchen status through event-driven updates. When orders are placed or completed, events automatically update the analytics dashboard that kitchen staff and managers use to track pending orders and revenue.

## Input Format
- operations: List[Tuple] - Restaurant operations to process
- Operations: ("PLACE_ORDER", order_id, customer_name, pizza_type, quantity), ("COMPLETE_ORDER", order_id), ("GET_PENDING_ORDERS"), ("GET_TOTAL_REVENUE"), ("GET_ORDER_COUNT")

## Output Format
- List containing results for query operations only

## Examples

Example 1:

Input:

```
operations = [
    ("PLACE_ORDER", "ORD001", "Alice", "Margherita", 2),
    ("GET_PENDING_ORDERS"),
    ("COMPLETE_ORDER", "ORD001"),
    ("GET_TOTAL_REVENUE")
]
```

Output:

```
[["ORD001"], 25.98]
```

Example 2:

Input:

```
operations = [
    ("PLACE_ORDER", "ORD001", "Bob", "Pepperoni", 1),
    ("PLACE_ORDER", "ORD002", "Carol", "Vegetarian", 3),
    ("GET_ORDER_COUNT"),
    ("COMPLETE_ORDER", "ORD001"),
    ("GET_PENDING_ORDERS"),
    ("GET_TOTAL_REVENUE")
]
```

Output:

```
[2, ["ORD002"], 15.99]
```


# Requirements

## Explicit Requirements

- Implement separate OrderProcessor class for write operations and OrderAnalytics class for read operations
- Order operations must trigger events that automatically update analytics
- No direct communication between OrderProcessor and OrderAnalytics classes
- Only completed orders contribute to revenue calculation
- Menu pricing: Margherita \$12.99, Pepperoni \$15.99, Vegetarian \$14.99, Hawaiian \$16.99

## Implicit Requirements

- Event system contains necessary data for analytics updates
- Query operations use cached pre-computed values
- Orders start "pending", move to "completed" when completed
- Analytics reflect real-time system state after each operation

## Function Signature

```python
def process_restaurant_operations(operations: List[Tuple]) -> List[Any]:
    pass

class OrderProcessor:
    def place_order(self, order_id: str, customer_name: str, pizza_type: str, quantity: int) -> None
    def complete_order(self, order_id: str) -> None

class OrderAnalytics:
    def get_pending_orders(self) -> List[str]
    def get_total_revenue(self) -> float
    def get_order_count(self) -> int
```

## Edge Cases

- Placing an order with duplicate ID raises an error
- Completing a non-existent order raises an error
- Completing an already completed order raises an error
- Invalid pizza type raises an error
- Zero or negative quantity raises an error
- Quantity exceeding limit raises an error
- Empty pending orders returns empty list
- Total revenue with no completed orders returns zero
- Order count with no orders returns zero
- Provide the output with 2 decimal places. Trailing zeroes after the decimal may be trimmed.

## Constraints

- Order IDs must be unique (raise ValueError for duplicates)
- Pizza types must match menu items exactly
- Cannot complete non-existent or already completed orders (raise ValueError)
- Quantities must be positive integers (1-5)
- Empty system returns appropriate default values
- The solution should be complete and have a driver function for execution.

In [ ]:
# code

"""CQRS restaurant order management system implementation."""

from typing import List, Tuple, Any, Dict


class EventBus:
    """Event bus for decoupling OrderProcessor and OrderAnalytics."""

    def __init__(self):
        """Initialize the event bus."""
        self._listeners = []

    def subscribe(self, listener):
        """Subscribe a listener to receive events."""
        self._listeners.append(listener)

    def publish(self, event_type: str, data: Dict):
        """Publish an event to all subscribed listeners."""
        for listener in self._listeners:
            listener.handle_event(event_type, data)


class OrderProcessor:
    """Handles write operations - placing and completing orders."""

    MENU_PRICES = {
        "Margherita": 12.99,
        "Pepperoni": 15.99,
        "Vegetarian": 14.99,
        "Hawaiian": 16.99,
    }

    def __init__(self, event_bus: EventBus):
        """Initialize the order processor with an event bus."""
        self._event_bus = event_bus
        self._orders = {}  # order_id -> order_data
        self._order_statuses = {}  # order_id -> "pending" or "completed"

    def place_order(
        self, order_id: str, customer_name: str, pizza_type: str, quantity: int
    ) -> None:
        """Place a new pizza order."""
        # Validate order ID uniqueness
        if order_id in self._orders:
            raise ValueError(f"Order {order_id} already exists")

        # Validate pizza type
        if pizza_type not in self.MENU_PRICES:
            raise ValueError(f"Invalid pizza type: {pizza_type}")

        # Validate quantity
        if not isinstance(quantity, int) or quantity <= 0 or quantity > 5:
            raise ValueError(
                "Quantity must be a positive integer between 1 and 5"
            )

        # Calculate total price
        total_price = self.MENU_PRICES[pizza_type] * quantity

        # Store order
        order_data = {
            "order_id": order_id,
            "customer_name": customer_name,
            "pizza_type": pizza_type,
            "quantity": quantity,
            "total_price": total_price,
        }

        self._orders[order_id] = order_data
        self._order_statuses[order_id] = "pending"

        # Publish event
        self._event_bus.publish("ORDER_PLACED", order_data)

    def complete_order(self, order_id: str) -> None:
        """Mark an order as completed."""
        # Validate order exists
        if order_id not in self._orders:
            raise ValueError(f"Order {order_id} does not exist")

        # Validate order is not already completed
        if self._order_statuses[order_id] == "completed":
            raise ValueError(f"Order {order_id} is already completed")

        # Update status
        self._order_statuses[order_id] = "completed"

        # Get order data for event
        order_data = self._orders[order_id].copy()

        # Publish event
        self._event_bus.publish("ORDER_COMPLETED", order_data)


class OrderAnalytics:
    """Handles read operations - analytics and queries."""

    def __init__(self, event_bus: EventBus):
        """Initialize analytics with event bus subscription."""
        self._event_bus = event_bus
        self._event_bus.subscribe(self)

        # Cached analytics data
        self._pending_orders = []  # List of order_ids
        self._total_revenue = 0.0
        self._order_count = 0

    def handle_event(self, event_type: str, data: Dict):
        """Handle events from OrderProcessor."""
        if event_type == "ORDER_PLACED":
            self._pending_orders.append(data["order_id"])
            self._order_count += 1

        elif event_type == "ORDER_COMPLETED":
            # Remove from pending orders
            if data["order_id"] in self._pending_orders:
                self._pending_orders.remove(data["order_id"])

            # Add to total revenue
            self._total_revenue += data["total_price"]

    def get_pending_orders(self) -> List[str]:
        """Get list of pending order IDs."""
        return self._pending_orders.copy()

    def get_total_revenue(self) -> float:
        """Get total revenue from completed orders."""
        return round(self._total_revenue, 2)

    def get_order_count(self) -> int:
        """Get total number of orders placed."""
        return self._order_count


def process_restaurant_operations(operations: List[Tuple]) -> List[Any]:
    """Process restaurant operations and return query results."""
    # Initialize event bus and components
    event_bus = EventBus()
    processor = OrderProcessor(event_bus)
    analytics = OrderAnalytics(event_bus)

    results = []

    for operation in operations:
        # Handle both tuple and string operations
        if isinstance(operation, str):
            op_type = operation
        else:
            op_type = operation[0]

        if op_type == "PLACE_ORDER":
            _, order_id, customer_name, pizza_type, quantity = operation
            processor.place_order(
                order_id, customer_name, pizza_type, quantity
            )

        elif op_type == "COMPLETE_ORDER":
            _, order_id = operation
            processor.complete_order(order_id)

        elif op_type == "GET_PENDING_ORDERS":
            results.append(analytics.get_pending_orders())

        elif op_type == "GET_TOTAL_REVENUE":
            results.append(analytics.get_total_revenue())

        elif op_type == "GET_ORDER_COUNT":
            results.append(analytics.get_order_count())

    return results


# Driver function for testing
def main():
    """Driver function to test the implementation."""
    # Test Example 1
    print("Example 1:")
    operations1 = [
        ("PLACE_ORDER", "ORD001", "Alice", "Margherita", 2),
        ("GET_PENDING_ORDERS"),
        ("COMPLETE_ORDER", "ORD001"),
        ("GET_TOTAL_REVENUE"),
    ]
    result1 = process_restaurant_operations(operations1)
    print(f"Input: {operations1}")
    print(f"Output: {result1}")
    print("Expected: [['ORD001'], 25.98]")
    print()

    # Test Example 2
    print("Example 2:")
    operations2 = [
        ("PLACE_ORDER", "ORD001", "Bob", "Pepperoni", 1),
        ("PLACE_ORDER", "ORD002", "Carol", "Vegetarian", 3),
        ("GET_ORDER_COUNT"),
        ("COMPLETE_ORDER", "ORD001"),
        ("GET_PENDING_ORDERS"),
        ("GET_TOTAL_REVENUE"),
    ]
    result2 = process_restaurant_operations(operations2)
    print(f"Input: {operations2}")
    print(f"Output: {result2}")
    print("Expected: [2, ['ORD002'], 15.99]")
    print()

    # Test edge cases
    print("Testing edge cases:")

    # Test empty system
    empty_ops = [
        ("GET_PENDING_ORDERS"),
        ("GET_TOTAL_REVENUE"),
        ("GET_ORDER_COUNT"),
    ]
    empty_result = process_restaurant_operations(empty_ops)
    print(f"Empty system: {empty_result}")
    print("Expected: [[], 0.0, 0]")
    print()

    # Test error cases
    try:
        # Duplicate order ID
        ops = [
            ("PLACE_ORDER", "ORD001", "Alice", "Margherita", 2),
            ("PLACE_ORDER", "ORD001", "Bob", "Pepperoni", 1),
        ]
        process_restaurant_operations(ops)
        print("ERROR: Should have raised ValueError for duplicate order")
    except ValueError as e:
        print(f"Correctly caught duplicate order error: {e}")

    try:
        # Invalid pizza type
        ops = [("PLACE_ORDER", "ORD001", "Alice", "InvalidPizza", 1)]
        process_restaurant_operations(ops)
        print("ERROR: Should have raised ValueError for invalid pizza type")
    except ValueError as e:
        print(f"Correctly caught invalid pizza error: {e}")

    try:
        # Invalid quantity
        ops = [("PLACE_ORDER", "ORD001", "Alice", "Margherita", 0)]
        process_restaurant_operations(ops)
        print("ERROR: Should have raised ValueError for invalid quantity")
    except ValueError as e:
        print(f"Correctly caught invalid quantity error: {e}")

    try:
        # Complete non-existent order
        ops = [("COMPLETE_ORDER", "NONEXISTENT")]
        process_restaurant_operations(ops)
        print("ERROR: Should have raised ValueError for non-existent order")
    except ValueError as e:
        print(f"Correctly caught non-existent order error: {e}")

    try:
        # Complete already completed order
        ops = [
            ("PLACE_ORDER", "ORD001", "Alice", "Margherita", 1),
            ("COMPLETE_ORDER", "ORD001"),
            ("COMPLETE_ORDER", "ORD001"),
        ]
        process_restaurant_operations(ops)
        print(
            "ERROR: Should have raised ValueError for already completed order"
        )
    except ValueError as e:
        print(f"Correctly caught already completed order error: {e}")


if __name__ == "__main__":
    main()


In [ ]:
# tests

"""Comprehensive test suite for CQRS restaurant order management system."""

import unittest
from main import process_restaurant_operations


class TestRestaurantOperations(unittest.TestCase):
    """Test cases for restaurant operations management system."""

    def test_example_1_from_problem(self):
        """Test the first example from problem statement."""
        operations = [
            ("PLACE_ORDER", "ORD001", "Alice", "Margherita", 2),
            ("GET_PENDING_ORDERS"),
            ("COMPLETE_ORDER", "ORD001"),
            ("GET_TOTAL_REVENUE"),
        ]
        result = process_restaurant_operations(operations)
        self.assertEqual(result, [["ORD001"], 25.98])

    def test_example_2_from_problem(self):
        """Test the second example from problem statement."""
        operations = [
            ("PLACE_ORDER", "ORD001", "Bob", "Pepperoni", 1),
            ("PLACE_ORDER", "ORD002", "Carol", "Vegetarian", 3),
            ("GET_ORDER_COUNT"),
            ("COMPLETE_ORDER", "ORD001"),
            ("GET_PENDING_ORDERS"),
            ("GET_TOTAL_REVENUE"),
        ]
        result = process_restaurant_operations(operations)
        self.assertEqual(result, [2, ["ORD002"], 15.99])

    def test_empty_system_queries(self):
        """Test query operations on empty system."""
        operations = [
            ("GET_PENDING_ORDERS"),
            ("GET_TOTAL_REVENUE"),
            ("GET_ORDER_COUNT"),
        ]
        result = process_restaurant_operations(operations)
        self.assertEqual(result, [[], 0.0, 0])

    def test_all_pizza_types_pricing(self):
        """Test each pizza type has correct pricing."""
        test_cases = [
            ("Margherita", 12.99, "ORD_MAR"),
            ("Pepperoni", 15.99, "ORD_PEP"),
            ("Vegetarian", 14.99, "ORD_VEG"),
            ("Hawaiian", 16.99, "ORD_HAW"),
        ]

        for pizza_type, expected_price, order_id in test_cases:
            operations = [
                ("PLACE_ORDER", order_id, "TestCustomer", pizza_type, 1),
                ("COMPLETE_ORDER", order_id),
                ("GET_TOTAL_REVENUE"),
            ]
            result = process_restaurant_operations(operations)
            self.assertEqual(
                result,
                [expected_price],
                f"Failed for {pizza_type}: expected {expected_price}, "
                f"got {result[0]}",
            )

    def test_quantity_multiplication(self):
        """Test revenue calculation with different quantities."""
        operations = [
            ("PLACE_ORDER", "ORD001", "Alice", "Margherita", 3),
            ("COMPLETE_ORDER", "ORD001"),
            ("GET_TOTAL_REVENUE"),
        ]
        result = process_restaurant_operations(operations)
        self.assertAlmostEqual(result[0], 38.97, places=2)

    def test_duplicate_order_id_raises_error(self):
        """Test that duplicate order IDs raise ValueError."""
        operations = [
            ("PLACE_ORDER", "ORD001", "Alice", "Margherita", 1),
            ("PLACE_ORDER", "ORD001", "Bob", "Pepperoni", 2),
        ]
        with self.assertRaises(ValueError):
            process_restaurant_operations(operations)

    def test_invalid_pizza_type_raises_error(self):
        """Test that invalid pizza type raises ValueError."""
        operations = [("PLACE_ORDER", "ORD001", "Alice", "InvalidPizza", 1)]
        with self.assertRaises(ValueError):
            process_restaurant_operations(operations)

    def test_complete_nonexistent_order_raises_error(self):
        """Test completing non-existent order raises ValueError."""
        operations = [("COMPLETE_ORDER", "NONEXISTENT")]
        with self.assertRaises(ValueError):
            process_restaurant_operations(operations)

    def test_complete_already_completed_order_raises_error(self):
        """Test completing already completed order raises ValueError."""
        operations = [
            ("PLACE_ORDER", "ORD001", "Alice", "Margherita", 1),
            ("COMPLETE_ORDER", "ORD001"),
            ("COMPLETE_ORDER", "ORD001"),
        ]
        with self.assertRaises(ValueError):
            process_restaurant_operations(operations)

    def test_zero_quantity_raises_error(self):
        """Test zero quantity raises ValueError."""
        operations = [("PLACE_ORDER", "ORD001", "Alice", "Margherita", 0)]
        with self.assertRaises(ValueError):
            process_restaurant_operations(operations)

    def test_negative_quantity_raises_error(self):
        """Test negative quantity raises ValueError."""
        operations = [("PLACE_ORDER", "ORD001", "Bob", "Pepperoni", -1)]
        with self.assertRaises(ValueError):
            process_restaurant_operations(operations)

    def test_quantity_exceeding_limit_raises_error(self):
        """Test quantity exceeding limit (5) raises ValueError."""
        operations = [("PLACE_ORDER", "ORD001", "Carol", "Vegetarian", 6)]
        with self.assertRaises(ValueError):
            process_restaurant_operations(operations)

    def test_valid_quantity_range(self):
        """Test valid quantity range (1-5)."""
        for quantity in range(1, 6):
            operations = [
                (
                    "PLACE_ORDER",
                    f"ORD{quantity}",
                    "Test",
                    "Margherita",
                    quantity,
                ),
                ("GET_ORDER_COUNT"),
            ]
            result = process_restaurant_operations(operations)
            self.assertEqual(result, [1])

    def test_pending_orders_dont_affect_revenue(self):
        """Test that only completed orders contribute to revenue."""
        operations = [
            ("PLACE_ORDER", "ORD001", "Alice", "Margherita", 2),
            ("PLACE_ORDER", "ORD002", "Bob", "Pepperoni", 1),
            ("GET_TOTAL_REVENUE"),
            ("COMPLETE_ORDER", "ORD001"),
            ("GET_TOTAL_REVENUE"),
            ("COMPLETE_ORDER", "ORD002"),
            ("GET_TOTAL_REVENUE"),
        ]
        result = process_restaurant_operations(operations)
        self.assertEqual(result, [0.0, 25.98, 41.97])

    def test_order_count_includes_all_orders(self):
        """Test order count includes both pending and completed orders."""
        operations = [
            ("PLACE_ORDER", "ORD001", "Alice", "Margherita", 1),
            ("PLACE_ORDER", "ORD002", "Bob", "Pepperoni", 1),
            ("GET_ORDER_COUNT"),
            ("COMPLETE_ORDER", "ORD001"),
            ("GET_ORDER_COUNT"),
        ]
        result = process_restaurant_operations(operations)
        self.assertEqual(result, [2, 2])

    def test_pending_orders_tracking(self):
        """Test pending orders are tracked correctly."""
        operations = [
            ("PLACE_ORDER", "ORD001", "Alice", "Margherita", 1),
            ("PLACE_ORDER", "ORD002", "Bob", "Pepperoni", 1),
            ("PLACE_ORDER", "ORD003", "Carol", "Hawaiian", 1),
            ("GET_PENDING_ORDERS"),
            ("COMPLETE_ORDER", "ORD002"),
            ("GET_PENDING_ORDERS"),
            ("COMPLETE_ORDER", "ORD001"),
            ("GET_PENDING_ORDERS"),
        ]
        result = process_restaurant_operations(operations)
        self.assertEqual(
            result,
            [["ORD001", "ORD002", "ORD003"], ["ORD001", "ORD003"], ["ORD003"]],
        )

    def test_complex_workflow(self):
        """Test complex workflow with multiple operations."""
        operations = [
            ("PLACE_ORDER", "ORD001", "Alice", "Margherita", 2),
            ("PLACE_ORDER", "ORD002", "Bob", "Hawaiian", 1),
            ("PLACE_ORDER", "ORD003", "Carol", "Vegetarian", 3),
            ("GET_ORDER_COUNT"),
            ("GET_PENDING_ORDERS"),
            ("COMPLETE_ORDER", "ORD001"),
            ("GET_TOTAL_REVENUE"),
            ("COMPLETE_ORDER", "ORD003"),
            ("GET_PENDING_ORDERS"),
            ("GET_TOTAL_REVENUE"),
            ("GET_ORDER_COUNT"),
        ]
        result = process_restaurant_operations(operations)
        expected = [
            3,
            ["ORD001", "ORD002", "ORD003"],
            25.98,
            ["ORD002"],
            70.95,
            3,
        ]
        self.assertEqual(result, expected)

    def test_decimal_precision(self):
        """Test decimal precision in revenue calculations."""
        operations = [
            ("PLACE_ORDER", "ORD001", "Alice", "Hawaiian", 5),
            ("COMPLETE_ORDER", "ORD001"),
            ("GET_TOTAL_REVENUE"),
        ]
        result = process_restaurant_operations(operations)
        self.assertEqual(result, [84.95])

    def test_no_write_operations_returns_empty_list(self):
        """Test that operations with only write commands return empty list."""
        operations = [
            ("PLACE_ORDER", "ORD001", "Alice", "Margherita", 1),
            ("COMPLETE_ORDER", "ORD001"),
        ]
        result = process_restaurant_operations(operations)
        self.assertEqual(result, [])


if __name__ == "__main__":
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: https://labeling-ms.turing.com/conversations/83202/view

#### Model code:

```python
from typing import List, Tuple, Any

# <Issue>: Missing event-driven architecture - no EventBus or decoupling
# between OrderProcessor and OrderAnalytics
MENU_PRICES = {
    "Margherita": 12.99,
    "Pepperoni": 15.99,
    "Vegetarian": 14.99,
    "Hawaiian": 16.99
}

class OrderProcessor:
    def __init__(self):
        self.orders = {}  # Store orders with their status
        self.completed_orders = {}
    
    def place_order(self, order_id: str, customer_name: str, pizza_type: str, quantity: int) -> Tuple[str, str, str, int]:
        # <Issue>: Wrong return type - should return None
        # <Issue>: Error message, Quantity validation doesn't match
        # specification
        if order_id in self.orders:
            raise ValueError(f"Duplicate order ID: {order_id}")
        if pizza_type not in MENU_PRICES:
            raise ValueError(f"Invalid pizza type: {pizza_type}")
        if not (1 <= quantity <= 5):
            raise ValueError(f"Invalid quantity: {quantity}")
        
        self.orders[order_id] = {"customer_name": customer_name, "pizza_type": pizza_type, "quantity": quantity, "status": "pending"}
        return order_id, customer_name, pizza_type, quantity
    
    def complete_order(self, order_id: str) -> Tuple[str, int]:
        if order_id not in self.orders:
            raise ValueError(f"Order ID {order_id} does not exist")
        if self.orders[order_id]["status"] == "completed":
            raise ValueError(f"Order ID {order_id} already completed")
        
        self.orders[order_id]["status"] = "completed"
        self.completed_orders[order_id] = self.orders[order_id]
        return order_id, self.completed_orders[order_id]["quantity"]
    
class OrderAnalytics:
    def __init__(self):
        self.pending_orders = []
        self.total_revenue = 0.0
        self.order_count = 0

    def update_order(self, order_id: str, status: str, pizza_type: str = None, quantity: int = 0):
        if status == "pending":
            self.pending_orders.append(order_id)
            self.order_count += 1
        elif status == "completed":
            if order_id in self.pending_orders:
                self.pending_orders.remove(order_id)
            if pizza_type:
                self.total_revenue += MENU_PRICES[pizza_type] * quantity
        
    def get_pending_orders(self) -> List[str]:
        return self.pending_orders
    
    def get_total_revenue(self) -> float:
        return round(self.total_revenue, 2)
    
    def get_order_count(self) -> int:
        return self.order_count

def process_restaurant_operations(operations: List[Tuple]) -> List[Any]:
    order_processor = OrderProcessor()
    order_analytics = OrderAnalytics()
    results = []
    
    for operation in operations:
        op_type = operation[0]
        
        if op_type == "PLACE_ORDER":
            order_id, customer_name, pizza_type, quantity = order_processor.place_order(*operation[1:])
            order_analytics.update_order(order_id, "pending")
        
        # <Issue>: Accessing internal state of OrderProcessor violates
        # encapsulation
        elif op_type == "COMPLETE_ORDER":
            order_id, quantity = order_processor.complete_order(*operation[1:])
            pizza_type = order_processor.orders[order_id]["pizza_type"]
            order_analytics.update_order(order_id, "completed", pizza_type, quantity)
        
        elif op_type == "GET_PENDING_ORDERS":
            results.append(order_analytics.get_pending_orders())
        
        elif op_type == "GET_TOTAL_REVENUE":
            results.append(order_analytics.get_total_revenue())
        
        elif op_type == "GET_ORDER_COUNT":
            results.append(order_analytics.get_order_count())
    
    return results

# Example usage:
operations1 = [
    ("PLACE_ORDER", "ORD001", "Alice", "Margherita", 2),
    ("GET_PENDING_ORDERS"),
    ("COMPLETE_ORDER", "ORD001"),
    ("GET_TOTAL_REVENUE")
]
print(process_restaurant_operations(operations1))  # Output: [["ORD001"], 25.98]

operations2 = [
    ("PLACE_ORDER", "ORD001", "Bob", "Pepperoni", 1),
    ("PLACE_ORDER", "ORD002", "Carol", "Vegetarian", 3),
    ("GET_ORDER_COUNT"),
    ("COMPLETE_ORDER", "ORD001"),
    ("GET_PENDING_ORDERS"),
    ("GET_TOTAL_REVENUE")
]
print(process_restaurant_operations(operations2))  # Output: [2, ["ORD002"], 15.99]
```